In [1]:
using Pkg
Pkg.activate("./Packages/Project.toml");

  Activating new project at `~/Repos/Heisenberg-Model-3D/Packages`


In [2]:
using Random

In [3]:
function RandomSpin()
    """Creates a random unit vector"""
    θ = rand(0:1e-6:π)
    ϕ = rand(0:1e-6:2π)
    return [sin(θ)*cos(ϕ), sin(θ)*sin(ϕ), cos(θ)]
end

function AcceptanceProbability(Spin1, Spin2, n, βJ)
    """Computes the probability of adding Spin2 - neighbour of
    Spin1 - to the cluster. It uses the energy βJ and the vector 
    of reflection n"""
    return 1 - exp(min(0,-2βJ*transpose(Spin1)*n*transpose(Spin2)*n))
end

function ReflectSpin(S,n,state)
    """Reflectes the spin state[S] around the plane defined by the
    normal vector n"""
    state[S] -=  2(transpose(state[S])*n)*n
end

function GetNeighbours(x,L,L2,L3)
    """Gets the coordinates of the 6 neighbour spins of the one in index x. It uses 
    periodic boundery conditions in a 3D lattice of length L (area L2, volume L3). 
    The matriz it returns contains the neighbours as folows: 
    [Up-Down-Left-Right-Front-Back]"""
    return [L2*((x-1)÷L2)+(x-1-L+L2)%L2+1,
            L2*((x-1)÷L2)+(x-1+L)%L2+1,
            L*((x-1)÷L)+(x+L-2)%L+1,
            L*((x-1)÷L)+(x)%L+1,
            (x-L2+L3-1)%L3+1,
            (x+L2-1)%L3+1]
end

function Grow_Reflect(S, Cluster, n, βJ, L, L2, L3, state)
    """Checks every neighbour of spin state[S] and adds them to the Cluster given
    a certain probability (See to AcceptanceProbability). Finally, it reflects
    the spin around the normal vector n (Refair to ReflectSpin)"""
    #Checks neighbours
    for Sn in GetNeighbours(S,L,L2,L3)
        if Sn ∉ Cluster && rand(0:1e-15:1) < AcceptanceProbability(state[S],state[Sn],n,βJ)
            #Adds neighbour to Cluster
            push!(Cluster,Sn)
            #Checks neighbours of neighbour
            Grow_Reflect(Sn, Cluster, n, βJ, L, L2, L3, state)
        end
    end
    #Reflects the spin
    ReflectSpin(S,n, state)
end

function NewState(state, L, L2, L3, βJ)
    """Chooses a randon spin from the state with lattice sice L, and also a
    random normal vector. From there it builds the cluster using Wolff algorithm
    (See Grow_Reflect)"""
    n = RandomSpin()     #Initial random Normal Vector
    #n = [0,0,1]         #Just for testing
    S0 = rand(1:1:L3)    #Initial random Spin
    Cluster = [S0]       #Stores the indexes of the spins added to the cluster
    Grow_Reflect(S0, Cluster, n, βJ, L, L2, L3, state) #Builds cluster and flips states for new state
    print("The Cluster contains:  ",Cluster,"\n") #Just for testing
end;

In [4]:
function RandomIsing()
    """Creates spins only in the z-axis (testing only)"""
    return [0, 0, rand(-1:2:1)]
end;

In [5]:
βJ = .5   #Energy of interaction (Quantified)
L = 3     #Lattice size
L2 = L*L  #Lattice Area
L3 = L*L2 #Lattice Volume
#state = [RandomIsing() for i in 1:L3] #Just for testing
state = [RandomSpin() for i in 1:L3]   #The 3D lattice is store as a 1D-Array
#print(state,"\n\n")
#Random.seed!(51)
NewState(state, L, L2, L3, βJ); #New state
#print("\n",state)

The Cluster contains:  [18, 12, 17]
